In [1]:
%cd ..

C:\Users\HP\Desktop\chatbot


In [4]:
import pathlib
import textwrap
import time
import os

import keys
import google.generativeai as genai
import google.ai.generativelanguage as glm

from IPython import display
from IPython.display import Markdown

os.environ["GOOGLE_API_KEY"]= keys.gemini_api_key
genai.configure(api_key= os.environ["GOOGLE_API_KEY"])

def to_markdown(text):
    return Markdown(textwrap.indent(text, '> ', predicate= lambda _ : True))

In [20]:
order_statuses= {
    "1142": "Shipped",
    "1906": "Processing",
    "0745": "Delivered"
}


def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    return order_statuses.get(order_id, "Order ID not found")


def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    if order_id in [i for i in order_statuses.keys()]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

In [22]:
model = genai.GenerativeModel(
    model_name= "gemini-1.5-flash",
    tools=[get_order_status, initiate_return]
)

In [24]:
# always use in chat mode for function calling
chat = model.start_chat(enable_automatic_function_calling=True)

In [26]:
response = chat.send_message("What is the status of order 1906?")

In [28]:
response.text

'The order is currently being processed. \n'

In [36]:
for content in chat.history:
    part= content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print("-"*100)

user -> {'text': 'What is the status of order 1906?'}
----------------------------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '1906'}}}
----------------------------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
----------------------------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
----------------------------------------------------------------------------------------------------


In [38]:
response = chat.send_message("I want to return order 0745 because it is defecctive.")

In [40]:
response.text

'OK. I have initiated a return for order 0745. \n'

In [42]:
for content in chat.history:
    part= content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print("-"*100)

user -> {'text': 'What is the status of order 1906?'}
----------------------------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '1906'}}}
----------------------------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
----------------------------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
----------------------------------------------------------------------------------------------------
user -> {'text': 'I want to return order 0745 because it is defecctive.'}
----------------------------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '0745', 'reason': 'defective'}}}
---------

# Sequential function calls/ nested function calls

In [46]:
rsponse = chat.send_message("Can you check the status of order 1906? If it's delivered, please initiate return as it was the wrong order.")

In [50]:
rsponse.text

"Order 1906 is still being processed. I can't initiate a return yet. I'll check back on it later. \n"

In [52]:
rsponse = chat.send_message("Can you check the status of order 0745? If it's delivered, please initiate return as it was the wrong order.")

In [54]:
rsponse.text

"Order 0745 has been delivered. I've initiated a return for you because it was the wrong order. \n"

In [64]:
def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been cancelled successfully."
        else:
            return f"Order {order_id} cannot be cancelled as it is already {order_statuses[order_id]}."
    else:
        return "Invalid Order ID / Order ID not found. Cannot cancel order"

In [66]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[get_order_status, initiate_return, cancel_order]
)

In [68]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [70]:
response = chat.send_message("Can you check the status of order 1906? if it is delivered, please initiate return as it was wrong order, else cancel the order")

In [72]:
response.text

'OK. I have cancelled order 1906. \n'

In [74]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'Can you check the status of order 1906? if it is delivered, please initiate return as it was wrong order, else cancel the order'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '1906'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'cancel_order', 'args': {'order_id': '1906'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'cancel_order', 'response': {'result': 'Order 1906 has been cancelled successfully.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'OK. I have cancelled order 1906. 

# Parallel function calling

In [77]:
import random

def update_shipping_address(order_id: str, new_address: str) -> str:
    """Updates the shipping address for a given order ID."""
    if order_id in [i for i in order_statuses.keys()]:
        return f"Shipping address for order {order_id} has been updated to: {new_address}."
    else:
        return "Order ID not found. Cannot update shipping address."


def track_shipment(tracking_number: str) -> str:
    """Tracks the shipment with the given tracking number"""
    statuses = [
        "Your order is sent for packaging",
        "Your order has departed.",
        "Your order has arrived"
    ]
    return statuses[random.randrange(0,3)]
    

def apply_discount(order_id: str, discount_code: str) -> str:
    """Applies a discount to the given order ID."""
    statuses = [
        "Discount of $2 has been applied.",
        "Discount code provided has expired",
        "Invalid discount code"
    ]
    return statuses[random.randrange(0,2)]
    

def change_payment_method(order_id: str, payment_method: str) -> str:
    """Changes the payment method for a given order ID."""
    statuses = [
        "Payment method not supported",
        "Payment method changed successfully."
        "Error occured. Please try again later."
    ]
    return statuses[random.randrange(0,2)]


def provide_invoice(order_id: str) -> str:
    """Provides an invoice for the given order ID."""
    statuses = [
        "Invoice has been sent to your mail.",
        "Unable to generate Invoice ."
        "Daily limit exceeded. Try again tomorrow."
    ]
    return statuses[random.randrange(0,2)]


def extend_warranty(order_id: str, years: int) -> str:
    """Extends the warranty for a given order ID."""
    statuses = [
        "Warranty extended successfully",
        "Duration too long. Unable to extended."
        "Not possible for this order ID."
    ]
    return statuses[random.randrange(0,2)]


def check_product_availability(product_id: str) -> str:
    """Checks the availability of a product with the given product ID."""
    statuses = [
        "Product available (in stock)",
        "Product not available (out of stock)",
        "product available (limited stock)"
    ]
    return statuses[random.randrange(0,2)]

## Not possible to do parallel function calling in automatic mode, we have to do it manually

In [80]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    tools= [
        get_order_status, initiate_return, cancel_order, update_shipping_address,
        track_shipment, apply_discount, change_payment_method, provide_invoice,
        extend_warranty, check_product_availability
    ]
)

In [82]:
chat = model.start_chat()

In [100]:
response = chat.send_message("What is the status of order 1142? Can you extend the warranty of order 1906?")

In [102]:
response.candidates

[index: 0
content {
  parts {
    function_call {
      name: "get_order_status"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "1142"
          }
        }
      }
    }
  }
  parts {
    text: "\n\n"
  }
  parts {
    function_call {
      name: "extend_warranty"
      args {
        fields {
          key: "years"
          value {
            number_value: 1
          }
        }
        fields {
          key: "order_id"
          value {
            string_value: "1906"
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
]

# we have 2 function calls

In [107]:
# just extract the function names that the model chose to call

for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

get_order_status(order_id=1142)
extend_warranty(order_id=1906, years=1.0)


In [111]:
responses = {
    "get_order_status": get_order_status(order_id="1142"),
    "extend_warranty": extend_warranty(order_id="1906", years="1.0")
}

In [113]:
# build the response parts

response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

In [115]:
responses = chat.send_message(response_parts)
print(responses.text)

The status of order 1142 is shipped. I have extended the warranty of order 1906. 

